In [1]:
%matplotlib inline
import matplotlib
matplotlib.use('agg')
from matplotlib import pyplot as plt
from metrab_utils import *

import pickle
import os
import tensorflow as tf

model_fold = '/disk/apps/metrabs/models'

models = os.listdir(model_fold)
print(models)

['metrabs_rn18_y4', 'metrabs_eff2l_y4_360', 'metrabs_eff2s_y4', 'metrabs_mob3s_y4t', 'server.log', 'metrabs_mob3s_y4', 'metrabs_eff2l_y4']


In [ ]:
# load videos!
import numpy as np
from tqdm import tqdm
import time
import cv2

vid_name_back = '/media/tobi/Elements/thesis_videos/gopro/25_exp.MP4'
vid_name_front = '/media/tobi/Elements/thesis_videos/alpha/25_exp.MP4'
vid_name_front_small = '/media/tobi/Elements/thesis_videos/alpha/small/all/25_exp.mp4'

vid_name = vid_name_front

WIDTH, HEIGHT = 1200, 600

#mod = 'metrabs_eff2l_y4_360' #ALL poses?
mod = 'metrabs_eff2l_y4'

print(mod)
vid_file = 'prezo/%s.mp4'%mod
doc_file = 'prezo/%s.txt'%mod
res_file = 'prezo/%s.pkl'%mod

t = time.time()
model = tf.saved_model.load(os.path.join(model_fold, mod))
t = time.time() - t

with open(doc_file, 'w') as fh:
    fh.write('load: %.2fs\n'%t)


cap = cv2.VideoCapture(vid_name)
vid_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

out_vid = cv2.VideoWriter(vid_file, cv2.VideoWriter_fourcc('M','J','P','G'), 30, (WIDTH, HEIGHT))

frames = []
bs = 64
results = []
ts = []
for i in tqdm(range(vid_len)):
    ret, frame = cap.read()
    if ret is None:
        break
    if i < 2*30*60:
        continue

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames.append(frame)
    
    if len(frames) == bs:
        t = time.time()
        pred = model.detect_poses_batched(np.array(frames))
        t = time.time() - t
        with open(doc_file, 'a') as fh:
            fh.write('%f\n'%(t*1000))

        results.append(pred)

        # draw to video!
        fig = plt.figure(figsize=(20, 10))

        t = time.time()
        for j, frame in enumerate(frames):
            visualize(
                frame, 
                pred['boxes'][j].numpy(),
                pred['poses3d'][j].numpy(),
                pred['poses2d'][j].numpy(),
                model.per_skeleton_joint_edges['smpl+head_30'].numpy(), fig)
            img = fig_to_np(fig)
            fig.clf()
            out_vid.write(img)
        plt.close('all')
        t = time.time()
        if len(results) >= 5:
            break

        frames = []
        
out_vid.release()
pickle.dump(results, open(res_file, 'wb'))

metrabs_eff2l_y4


In [ ]:
frame